In [1]:
import numpy as np
from scipy.signal import convolve
from astropy.table import Table


In [2]:
# old stuff

region_ra, region_dec = (260.059728, 57.921219)
region_radius = 1
radii = [0.316, 0.1, 0.0316, 0.01]
pm_spacing = np.array(radii) * 10
num_cones = 10


minimum_count = 2
sigma_threshhold = 3
pm_threshhold = 5

# standard paths
infile = f'regions/region_ra{round(region_ra*100)}_dec{round(region_dec*100)}_rad{round(region_radius*100)}.vot'
outfile = f'region_candidates/region_ra{round(region_ra*100)}_dec{round(region_dec*100)}_rad{round(region_radius*100)}_candidates.txt'

with open(outfile, 'a') as fil:
    fil.write(f'# successful candidates for region at ({region_ra}, {region_dec}) and radius {region_radius}')

# first try to find file
try:
    gaia_table = Table.read(infile, format='votable')
    print(f"Table loaded from: {infile}")
    print(f"Number of objects: {len(gaia_table)}")
except FileNotFoundError:
    job = gaia_region_search(region_ra, region_dec, outfile=infile, radius=region_radius)
    gaia_table = job.get_results()
    print("Finished querying Gaia")
    gaia_table = gaia_table[outside_of_galactic_plane(gaia_table['ra'], gaia_table['dec'])]
    print("Finished filtering Gaia table")
    gaia_table['x'], gaia_table['y'] = azimuthal_equidistant_coordinates(gaia_table, region_ra, region_dec)
    print("Finished calculating x-y values done")
    print(f"Table dumped to: {infile}")
    print(f"Number of objects: {len(gaia_table)}")
    gaia_table.write(infile, overwrite='True', format='votable')

Table loaded from: regions/region_ra26006_dec5792_rad100.vot
Number of objects: 5331


In [4]:
min_radius = min(radii)
min_pm_spacing = min(pm_spacing)

bins = [region_radius//min_radius]*2 + [pm_threshhold//min_pm_spacing]*2
data_4d = np.array([gaia_table['x'], gaia_table['y'], gaia_table['ra'], gaia_table['dec']]).T
histo, edges = np.histogramdd(data_4d, bins=bins)
print(histo.shape)

# put bins in degrees
edges[0] *= 180/np.pi
edges[1] *= 180/np.pi

# set bins for plotting
X, Y = np.meshgrid(edges[0], edges[1])
print(X.shape)
# histo_mask = np.less(X[:-1, :-1]**2 + Y[:-1, :-1]**2, region_radius**2)

# convolve the histogram with different size tophats
convolved_data = []
for radius, pm_space in zip(radii, pm_spacing):
    spatial_size = radius//min_radius
    pm_size = pm_space//min_pm_spacing
    convolved_array = convolution.convolve(histo, convolution_kernel)
    convolved_data.append((radius, convolved_array))
    print(f"finished {radius}")


(99, 99, 49, 49)
(100, 100)


NameError: name 'convolution' is not defined

In [38]:
spa_dim = 2
pm_dim = 2
a = np.ones((spa_dim*2+1,spa_dim*2+1,pm_dim*2+1,pm_dim*2+1))
for i in np.arange(-spa_dim,1+spa_dim):
    for j in np.arange(-spa_dim,spa_dim+1):
        for k in np.arange(-pm_dim, pm_dim+1):
            for l in np.arange(-pm_dim, pm_dim+1):
                if (i/spa_dim)**2 + (j/spa_dim)**2 + (k/pm_dim)**2 + (l/pm_dim)**2 > 1:
                    a[i+spa_dim,j+spa_dim,k+pm_dim,l+pm_dim] = 0
a

array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 1., 1., 1.,

In [12]:
b = np.random.randint(-2, 2, (2,2,3,3))
print(b)
print(b[0])
a,c,d,f = b

[[[[-2  1  0]
   [ 1  1 -1]
   [ 0 -2 -1]]

  [[-1  0  0]
   [ 1  0  1]
   [ 0 -1 -2]]]


 [[[-2 -2 -2]
   [ 0  1  1]
   [-2  0  1]]

  [[-2  0  1]
   [ 1  0  0]
   [ 1  1  1]]]]
[[[-2  1  0]
  [ 1  1 -1]
  [ 0 -2 -1]]

 [[-1  0  0]
  [ 1  0  1]
  [ 0 -1 -2]]]


ValueError: not enough values to unpack (expected 4, got 2)